### installation

In [ ]:
#python 3.8 change it in anonconda environment create new one and install jupyter for it and open
!pip3 install Flask
!pip3 install Werkzeug
!pip install flask_cors
!pip install json
!pip install torchaudio
!pip install torch
!pip install denoiser
!pip install deepspeech
!pip install ffmpeg #add path in variable environment windows
!pip install pydub

# before pip install flask_classful and restart jupyter
# Flask-SocketIO==4.3.1
# python-engineio==3.13.2
# python-socketio==4.6.0
# Flask==2.0.3
# Werkzeug==2.0.3

!pip install keras
!pip install tensorflow

### Import  libraries

In [1]:
import json                            #deal with json files
from IPython import display as disp    # Required for denoiser Pretraind model [denoiser]
import torch                           # Required for denoiser Pretraind model [denoiser]
import torchaudio                      # Required for denoiser Pretraind model [denoiser]
from denoiser import pretrained        # Required for denoiser Pretraind model [denoiser]
from denoiser.dsp import convert_audio # Required for denoiser Pretraind model [denoiser]
from flask import Flask,request        # api
from flask_classful import FlaskView, route #api class
from flask_cors import CORS, cross_origin
from pydub import AudioSegment
import librosa
import wave
from keras import models
import numpy as np
import tensorflow as tf

### Init app for creating api functions

In [2]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

### deep speech api functions

In [3]:
class DeepSpeech(FlaskView):
    def __init__(self):
        self.deep_speech_pb=""
        self.deep_speech_scorer=""
        self.recorded_wav=""
        self.denoised_wav=""
        self.json_path=""
    @route("/path")
    def set_paths(self):
        paths = json.loads(request.data)
        self.deep_speech_pb=paths["pb_model"]
        self.deep_speech_scorer=paths["scorer_model"]
        self.recorded_wav=paths["record"]
        self.denoised_wav=paths["denoised_record"]
        self.json_path=paths["output_json"]
        self.meta_data=paths["meta_data"]
        self.generated_wav=paths["hkm_record"]
        self.root=paths["root"]
    def get_root_path(self):
        return self.root
    def get_record_path(self):
        return self.recorded_wav
    
    @route("/denoiser")
    def denoise_record(self):
        model = pretrained.dns64()
        wav, sr = torchaudio.load(self.recorded_wav)
        wav = convert_audio(wav, sr, model.sample_rate, model.chin)
        with torch.no_grad():
            denoised = model(wav[None])[0]
        x=disp.Audio(denoised.data.cpu().numpy(), rate=model.sample_rate)
        with open(self.denoised_wav, 'wb') as f:
            f.write(x.data)
    
    
    @route("/json")
    def generate_words(self):
        !deepspeech --json --model {self.deep_speech_pb} --scorer {self.deep_speech_scorer} --audio {self.denoised_wav} >{self.json_path}
    def get_words_without_taskeel(self):
        right_words=[]
        arabic_alphapet="ٱأإابتثجحخدذرزسشصضطظعغفقكلمنهويةىآؤءئ"
        with open(self.json_path) as f:
            data = json.load(f)
        transcripts=data['transcripts']
        word_without_taskel=""
        for word in transcripts[0]['words']:
                word_without_taskel=""
                for letter in word["word"]:
                    for alpha in arabic_alphapet:
                        if(letter==alpha):
                            word_without_taskel=word_without_taskel+letter
                right_words.append({"word":word_without_taskel,"start_time":word["start_time"],
                                   "duration":word["duration"]})
        for word in right_words:
            print(word["word"]," ",word["start_time"]," ",word["duration"])
        return right_words
    def get_hkm_words(self):
        hkm_words=[]
        with open(self.meta_data) as f:
            data = json.load(f)
        transcripts=data['words']
        hkm_indexes=data["indexes"]
        for idx in hkm_indexes:
            hkm_words.append(transcripts[idx]["word"])
        return hkm_words
    
        
    def get_hkm_time(self):
        right_words=self.get_words_without_taskeel()
        hkm_words=self.get_hkm_words()
        print("hkm klmat")
        for word in hkm_words:
            print(word)
        idx=0
        start_time=0.0
        duration=0.0
        for word in right_words:
            if(hkm_words[idx]==word["word"] and idx==0):
                start_time=word["start_time"]
                duration=word["duration"]
                idx=idx+1
            elif(hkm_words[idx]==word["word"] and idx!=0):
                duration=duration+word["duration"]
                idx=idx+1
            else:
                idx=0
            if(len(hkm_words)==idx):
                    break;
        ans=[-1.0,-1.0]
        if(idx==len(hkm_words)):
            print("start_time:",start_time)
            print("duration:",duration)
            ans[0]=start_time
            ans[1]=duration
        return ans
    @route("/hkm_record")
    def generate_hkm_record(self,ans):
        start_time=ans[0]
        duration=ans[1]
        !ffmpeg -ss "$start_time" -t  "$duration" -i "$self.denoised_wav" "$self.generated_wav" -y
    @route("/deep_speech")
    def run_deep_speech(self):
        self.denoise_record()
        self.generate_words()
        ans=self.get_hkm_time()
        if(ans[0]==-1):
            return "0"
        self.generate_hkm_record(ans)
        return "1"
        

In [4]:
class RuleModel(FlaskView):
    def init(self):
        self.model_path=""
        self.record_path=""
    def set_paths(self,paths):
        self.model_path=paths["rule_path"]
        self.record_path=paths["hkm_record"]
        self.x=paths["model_x"]
    
    def fixing_signal_range (self,file_path , signal ) :
        print(self.x)
        print(type(self.x))
        rate =  self.get_duration(file_path) /self.x;
        new_sound_1 = librosa.effects.time_stretch(signal, rate=rate)
        return new_sound_1
    
    
    @route("/rule")
    def run_model(self):
        SAMPLE_RATE = 22050
        num_mfcc=13 
        n_fft=2048
        hop_length=512
        file_path = self.record_path
        signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
        signal = self.fixing_signal_range (file_path , signal )
        mfcc = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
        mfcc = mfcc.T
        return self.runModel(mfcc)
    def get_duration(self,file_path):
        with wave.open(file_path) as mywav:
            duration = mywav.getnframes() / mywav.getframerate()
        return duration

    def runModel (self,mfccs) :
        model = models.load_model(self.model_path)
        print(self.model_path)
        mfccs = np.array(mfccs)
        X_train2 = []
        temp = np.array(mfccs).flatten()
        X_train2.append(temp)
        mfccs = X_train2 
        mfccs = tf.stack(mfccs)
        y_pred = model.predict(mfccs)
        if (np.round(y_pred[0]) == 0 ):
            return "1"
        else:
            return "0"

In [5]:
class Record(FlaskView):
    def __init__(self):
        self.deep_speech=DeepSpeech()
        self.rule_model=RuleModel()
    @route('/<chunck>', methods=['POST'])
    @cross_origin()
    def check(self,chunck):
        x="0"
        if(len(request.data)<800):
            self.deep_speech.set_paths()
            data = json.loads(request.data)
            self.rule_model.set_paths(data)
        else:
            temp_filename = self.deep_speech.get_root_path()+"/records/tmp.wav"
            with open(temp_filename, '+wb') as f:
                f.write(request.get_data())
            audio = AudioSegment.from_file(temp_filename, format="webm")
            output_filename = self.deep_speech.get_record_path()
            audio.export(output_filename, format="wav")
            x=self.deep_speech.run_deep_speech()
            print(x)
            if(x=="1"):
                x=self.rule_model.run_model()
        return x

In [6]:
@app.route('/read_file', methods = ['POST','GET'])
@cross_origin()
def api_message():
    with open(request.data) as f:
        data = json.load(f)
    transcripts=data['words']
    result=""
    for word in transcripts:
        result=result+word["word"]+" "
    return result

In [ ]:
deepspeech=DeepSpeech()
record=Record()
rule_model=RuleModel()
deepspeech.register(app,route_base = '/')
record.register(app,route_base = '/records')
rule_model.register(app,route_base = '/rules')
if __name__ == '__main__':
    app.run(debug= False , host= 'localhost',port=2000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:2000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/May/2023 21:08:40] "POST /read_file HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 21:09:18] "OPTIONS /records/save HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 21:09:18] "OPTIONS /records/save HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 21:09:18] "POST /records/save HTTP/1.1" 200 -
TensorFlow: v2.3.0-6-g23ad988fcd
DeepSpeech: v0.9.3-0-gf2e9c858
Loading model from file D:/web/deep_speech_models/output.pb
Loaded model in 0.206s.
Loading scorer from files D:/web/deep_speech_models/quran.scorer
Loaded scorer in 0.0185s.
Running inference.
Inference took 2.323s for 6.120s audio file.
127.0.0.1 - - [16/May/2023 21:09:26] "POST /records/save HTTP/1.1" 200 -


قال   1.2   0.4
إن   1.62   0.68
ٱلله   2.34   0.5
لا   2.92   0.36
تثريب   3.3   0.86
وجعلنى   4.2   0.8
نبيا   5.02   1.08
hkm klmat
عبد
ٱلله
0


127.0.0.1 - - [16/May/2023 21:09:55] "POST /read_file HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 21:10:02] "OPTIONS /records/save HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 21:10:02] "OPTIONS /records/save HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 21:10:02] "POST /records/save HTTP/1.1" 200 -


قل   1.24   0.14
هو   1.4   0.28
ٱلله   1.8   0.46
أحد   2.34   1.0
hkm klmat
هو
ٱلله
start_time: 1.4
duration: 0.74


...

127.0.0.1 - - [16/May/2023 21:21:31] "OPTIONS /records/save HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 21:21:31] "OPTIONS /records/save HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 21:21:31] "POST /records/save HTTP/1.1" 200 -


أم   0.84   0.2
ٱعبدوا   1.06   0.48
ٱلله   1.58   0.66
وٱتقوه   2.26   1.0
وأطيعون   3.28   1.62
hkm klmat
ٱعبدوا
ٱلله
start_time: 1.06
duration: 1.1400000000000001


TensorFlow: v2.3.0-6-g23ad988fcd
DeepSpeech: v0.9.3-0-gf2e9c858
Loading model from file D:/web/deep_speech_models/output.pb
Loaded model in 0.103s.
Loading scorer from files D:/web/deep_speech_models/quran.scorer
Loaded scorer in 0.00195s.
Running inference.
Inference took 2.200s for 4.920s audio file.
ffmpeg version 6.0-full_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enab

1
4
<class 'int'>
D:/web/hkm1/model.h5
1/1 [==============================] - 0s 61ms/step


127.0.0.1 - - [16/May/2023 21:21:37] "POST /records/save HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 21:39:12] "POST /read_file HTTP/1.1" 200 -
